In [1]:
import pickle
import pandas as pd

In [2]:
# ниже подготовка к записи данных в базу

In [3]:
with open('../output_products.pkl', 'rb') as f:
    data = pickle.load(f)

In [4]:
pd.DataFrame(data).isnull().mean().index

Index(['name', 'product_url', 'offer_count', 'min_price', 'Производитель',
       'Тип загрузки', 'Максимальная загрузка белья, кг', 'Тип машины',
       'Глубина, см', 'Ширина, см', 'Высота, см', 'Вес, кг',
       'Степень автоматизации', 'Количество программ', 'Дисплей', 'Установка',
       'Цвет', 'Класс стирки', 'Дополнительное полоскание',
       'Максимальное количество оборотов отжима, об/мин', 'Класс отжима',
       'Выбор скорости отжима', 'Сушка', 'Класс энергопотребления',
       'Материал бака', 'Дополнительные функции',
       'Возможность дозагрузки белья', 'Отмена отжима',
       'Программа «легкая глажка»', 'Прямой привод (direct drive)',
       'Инверторный двигатель', 'Безопасность', 'Расход воды за стирку, л',
       'Максимальная загрузка белья для сушки, кг', 'Малютка'],
      dtype='object')

In [1]:
from langchain_community.llms import LlamaCpp
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [2]:
chat_model = LlamaCpp(
    # llama 13b saiga: -- '../models/model-q4_K(2).gguf'
    # roleplay - mixtral moe 8x7b: -- mixtral-8x7b-moe-rp-story.Q4_K_M
    # mixtral-8x7b-v0.1.Q4_K_M
    model_path='/home/amstel/llm/models/saiga_mistral_7b.gguf',
    n_gpu_layers=20,  # 28 for llama2 13b, 10 for mixtral
    max_tokens=2048,
    n_batch=16,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    verbose=True,
    temperature=0.0,
)

context = """'Производитель','Тип загрузки', 'Максимальная загрузка белья, кг', 'Тип машины',
       'Глубина, см', 'Ширина, см', 'Высота, см', 'Вес, кг',
       'Степень автоматизации', 'Количество программ', 'Дисплей', 'Установка',
       'Цвет', 'Класс стирки', 'Дополнительное полоскание',
       'Максимальное количество оборотов отжима, об/мин', 'Класс отжима',
       'Выбор скорости отжима', 'Сушка', 'Класс энергопотребления',
       'Материал бака', 'Дополнительные функции',
       'Возможность дозагрузки белья', 'Отмена отжима',
       'Программа «легкая глажка»', 'Прямой привод (direct drive)',
       'Инверторный двигатель', 'Безопасность', 'Расход воды за стирку, л',
       'Максимальная загрузка белья для сушки, кг', 'Малютка'"""

raw_template = f'''<s> [INST] You are given a list of properties names for the washing machine. Come up with appropraite field names for an SQL database. Return a JSON mapping descriptions to short aliases. [/INST]
Names: {context}
JSON: 
'''

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /home/amstel/llm/models/saiga_mistral_7b.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u3

In [3]:
class JsonMap(BaseModel):
    source_name: str = Field(description="source name in Russian")
    sql_alias: str = Field(description="short sql alias")

In [4]:
template = ChatPromptTemplate.from_template(raw_template)
parser = JsonOutputParser(pydantic_object=JsonMap)
chain = template | chat_model | parser

print(chain.invoke(input={}))


llama_print_timings:        load time =    1127.44 ms
llama_print_timings:      sample time =     155.74 ms /   534 runs   (    0.29 ms per token,  3428.73 tokens per second)
llama_print_timings: prompt eval time =   21830.90 ms /   390 tokens (   55.98 ms per token,    17.86 tokens per second)
llama_print_timings:        eval time =   62798.93 ms /   533 runs   (  117.82 ms per token,     8.49 tokens per second)
llama_print_timings:       total time =   86175.64 ms /   923 tokens


{'Производитель': 'Producer', 'Тип загрузки': 'LoadingType', 'Максимальная загрузка белья, кг': 'MaxLoad', 'Тип машины': 'MachineType', 'Глубина, см': 'Depth', 'Ширина, см': 'Width', 'Высота, см': 'Height', 'Вес, кг': 'Weight', 'Степень автоматизации': 'AutomationDegree', 'Количество программ': 'ProgramsCount', 'Дисплей': 'Display', 'Установка': 'Installation', 'Цвет': 'Color', 'Класс стирки': 'WashingClass', 'Дополнительное полоскание': 'AdditionalRinse', 'Максимальное количество оборотов отжима, об/мин': 'MaxSpinningSpeed', 'Класс отжима': 'SpinningClass', 'Выбор скорости отжима': 'ChooseSpinningSpeed', 'Сушка': 'Drying', 'Класс энергопотребления': 'EnergyClass', 'Материал бака': 'TubMaterial', 'Дополнительные функции': 'AdditionalFunctions', 'Возможность дозагрузки белья': 'PossibleOverload', 'Отмена отжима': 'CancelSpinning', 'Программа «легкая глажка»': 'LightWashProgram', 'Прямой привод (direct drive)': 'DirectDrive', 'Инверторный двигатель': 'InverterMotor', 'Безопасность': 'Saf

In [5]:
{
    "Производитель": "Manufacturer",
    "Тип загрузки": "LoadingType",
    "Максимальная загрузка белья, кг": "MaxLoad",
    "Тип машины": "MachineType",
    "Глубина, см": "Depth",
    "Ширина, см": "Width",
    "Высота, см": "Height",
    "Вес, кг": "Weight",
    "Степень автоматизации": "AutomationDegree",
    "Количество программ": "ProgramsCount",
    "Дисплей": "Display",
    "Установка": "Installation",
    "Цвет": "Color",
    "Класс стирки": "WashingClass",
    "Дополнительное полоскание": "AdditionalRinse",
    "Максимальное количество оборотов отжима, об/мин": "MaxSpinningSpeed",
    "Класс отжима": "SpinningClass",
    "Выбор скорости отжима": "ChooseSpinningSpeed",
    "Сушка": "Drying",
    "Класс энергопотребления": "EnergyClass",
    "Материал бака": "TubMaterial",
    "Дополнительные функции": "AdditionalFunctions",
    "Возможность дозагрузки белья": "PossibleOverload",
    "Отмена отжима": "CancelSpinning",
    "Программа «легкая глажка»": "LightWashProgram",
    "Прямой привод (direct drive)": "DirectDrive",
    "Инверторный двигатель": "InverterMotor",
    "Безопасность": "Safety",
    "Расход воды за стирку, л": "WaterConsumption",
    "Максимальная загрузка белья для сушки, кг": "MaxDryLoad",
    "Малютка": "Mop"
}


{'Производитель': 'Producer',
 'Тип загрузки': 'LoadingType',
 'Максимальная загрузка белья, кг': 'MaxLoad',
 'Тип машины': 'MachineType',
 'Глубина, см': 'Depth',
 'Ширина, см': 'Width',
 'Высота, см': 'Height',
 'Вес, кг': 'Weight',
 'Степень автоматизации': 'AutomationDegree',
 'Количество программ': 'ProgramsCount',
 'Дисплей': 'Display',
 'Установка': 'Installation',
 'Цвет': 'Color',
 'Класс стирки': 'WashingClass',
 'Дополнительное полоскание': 'AdditionalRinse',
 'Максимальное количество оборотов отжима, об/мин': 'MaxSpinningSpeed',
 'Класс отжима': 'SpinningClass',
 'Выбор скорости отжима': 'ChooseSpinningSpeed',
 'Сушка': 'Drying',
 'Класс энергопотребления': 'EnergyClass',
 'Материал бака': 'TubMaterial',
 'Дополнительные функции': 'AdditionalFunctions',
 'Возможность дозагрузки белья': 'PossibleOverload',
 'Отмена отжима': 'CancelSpinning',
 'Программа «легкая глажка»': 'LightWashProgram',
 'Прямой привод (direct drive)': 'DirectDrive',
 'Инверторный двигатель': 'InverterMo

In [ ]:
# import pandas as pd
# from sqlalchemy import create_engine
# from loguru import logger

# def select_data(
#     table: str,
#     where: str = None
# ):

#     #######################
#     user = 'scraperuser'
#     password = 'scraperpassword'
#     host = 'localhost'
#     port = '6432'
#     database = 'scraperdb'
#     connection_str = f'postgresql://{user}:{password}@{host}:{port}/{database}'
#     engine = create_engine(connection_str)
#     sql_from_table = 'scraped_data.product_item_list'
#     where_clause = 'crawl_id <= 2'
#     #######################

#     if where_clause:
#         sql_str = f'select * from {table} where {where};'
#     else:
#         sql_str = f'select * from {table};'
#     try:
#         with engine.connect() as connection_str:
#             print('Successfully connected to the PostgreSQL database')
#             df = pd.read_sql(
#                 sql = sql_str,
#                 con=connection_str, 
#             )
#             logger.warning(df.shape)
#         return df
#     except Exception as ex:
#         print(f'Sorry failed to connect: {ex}')
#         return pd.DataFrame()



# df = select_data(table=sql_from_table, where=where_clause)

In [ ]:
with open('')

In [14]:
for x in df['product_url'].tolist():
    if ' ' in x:
        print(x)

In [ ]:
# import pickle
# import pandas as pd
# from sqlalchemy import create_engine

# with open('./output.pkl', 'rb') as f:
#     data = pickle.load(f)
# df = pd.DataFrame(data[-1][0])
# df['crawl_id'] = 1

# user = 'scraperuser'
# password = 'scraperpassword'
# host = 'localhost'
# port = '6432'
# database = 'scraperdb'
# # for creating connection string
# connection_str = f'postgresql://{user}:{password}@{host}:{port}/{database}'
# # SQLAlchemy engine
# engine = create_engine(connection_str)
# # you can test if the connection is made or not
# try:
#     with engine.connect() as connection_str:
#         print('Successfully connected to the PostgreSQL database')
#         df.to_sql(
#             name='product_item_list',
#             con=connection_str, 
#             schema='scrape_results',
#             index=False,
#             if_exists='append',
#         )
# except Exception as ex:
#     print(f'Sorry failed to connect: {ex}')

,product_url,product_name,product_position,product_type_url,product_type_name,crawl_id
0,https://shop.by/stiralnye_mashiny/atlant_sma_6...,Стиральная машина ATLANT СМА 60У1214-01,1,https://shop.by/stiralnye_mashiny/?page_id=1,Стиральные машины,1
1,https://shop.by/stiralnye_mashiny/indesit_iwsb...,Стиральная машина Indesit IWSB 51051 BY,2,https://shop.by/stiralnye_mashiny/?page_id=1,Стиральные машины,1
2,https://shop.by/stiralnye_mashiny/atlant_sma_5...,Стиральная машина ATLANT СМА 50У107-000,3,https://shop.by/stiralnye_mashiny/?page_id=1,Стиральные машины,1
3,https://shop.by/stiralnye_mashiny/atlant_sma_6...,Стиральная машина ATLANT СМА 60У1010-00,4,https://shop.by/stiralnye_mashiny/?page_id=1,Стиральные машины,1
4,https://shop.by/stiralnye_mashiny/lg_f2v5gs0w/,Стиральная машина LG F2V5GS0W,5,https://shop.by/stiralnye_mashiny/?page_id=1,Стиральные машины,1
5,https://shop.by/stiralnye_mashiny/lg_f2j3ws2w/,Стиральная машина LG F2J3WS2W,6,https://shop.by/stiralnye_mashiny/?page_id=1,Стиральные машины,1
6,https://shop.by/stiralnye_mashiny/indesit_iwub...,Стиральная машина Indesit IWUB 4085,7,https://shop.by/stiralnye_mashiny/?page_id=1,Стиральные машины,1
7,https://shop.by/stiralnye_mashiny/candy_aqua_1...,Стиральная машина Candy AQUA 114D2-07,8,https://shop.by/stiralnye_mashiny/?page_id=1,Стиральные машины,1
8,https://shop.by/stiralnye_mashiny/indesit_iwub...,Стиральная машина Indesit IWUB 41051 BY,9,https://shop.by/stiralnye_mashiny/?page_id=1,Стиральные машины,1
9,https://shop.by/stiralnye_mashiny/lg_f2v3gs6w/,Стиральная машина LG F2V3GS6W,10,https://shop.by/stiralnye_mashiny/?page_id=1,Стиральные машины,1


In [15]:
pd.DataFrame.from_records(
    data=data[-1][0],
    # orient=''

)

,product_url,product_name,product_position,product_type_url,product_type_name
0,https://shop.by/stiralnye_mashiny/atlant_sma_6...,Стиральная машина ATLANT СМА 60У1214-01,1,https://shop.by/stiralnye_mashiny/?page_id=1,Стиральные машины
1,https://shop.by/stiralnye_mashiny/indesit_iwsb...,Стиральная машина Indesit IWSB 51051 BY,2,https://shop.by/stiralnye_mashiny/?page_id=1,Стиральные машины
2,https://shop.by/stiralnye_mashiny/atlant_sma_5...,Стиральная машина ATLANT СМА 50У107-000,3,https://shop.by/stiralnye_mashiny/?page_id=1,Стиральные машины
3,https://shop.by/stiralnye_mashiny/atlant_sma_6...,Стиральная машина ATLANT СМА 60У1010-00,4,https://shop.by/stiralnye_mashiny/?page_id=1,Стиральные машины
4,https://shop.by/stiralnye_mashiny/lg_f2v5gs0w/,Стиральная машина LG F2V5GS0W,5,https://shop.by/stiralnye_mashiny/?page_id=1,Стиральные машины
5,https://shop.by/stiralnye_mashiny/lg_f2j3ws2w/,Стиральная машина LG F2J3WS2W,6,https://shop.by/stiralnye_mashiny/?page_id=1,Стиральные машины
6,https://shop.by/stiralnye_mashiny/indesit_iwub...,Стиральная машина Indesit IWUB 4085,7,https://shop.by/stiralnye_mashiny/?page_id=1,Стиральные машины
7,https://shop.by/stiralnye_mashiny/candy_aqua_1...,Стиральная машина Candy AQUA 114D2-07,8,https://shop.by/stiralnye_mashiny/?page_id=1,Стиральные машины
8,https://shop.by/stiralnye_mashiny/indesit_iwub...,Стиральная машина Indesit IWUB 41051 BY,9,https://shop.by/stiralnye_mashiny/?page_id=1,Стиральные машины
9,https://shop.by/stiralnye_mashiny/lg_f2v3gs6w/,Стиральная машина LG F2V3GS6W,10,https://shop.by/stiralnye_mashiny/?page_id=1,Стиральные машины


[{'product_url': 'https://shop.by/stiralnye_mashiny/atlant_sma_60u1214_01/',
  'product_name': 'Стиральная машина ATLANT СМА 60У1214-01',
  'product_position': '1',
  'product_type_url': 'https://shop.by/stiralnye_mashiny/?page_id=1',
  'product_type_name': 'Стиральные машины'},
 {'product_url': 'https://shop.by/stiralnye_mashiny/indesit_iwsb_51051_by/',
  'product_name': 'Стиральная машина Indesit IWSB 51051 BY',
  'product_position': '2',
  'product_type_url': 'https://shop.by/stiralnye_mashiny/?page_id=1',
  'product_type_name': 'Стиральные машины'},
 {'product_url': 'https://shop.by/stiralnye_mashiny/atlant_sma_50u107/',
  'product_name': 'Стиральная машина ATLANT СМА 50У107-000',
  'product_position': '3',
  'product_type_url': 'https://shop.by/stiralnye_mashiny/?page_id=1',
  'product_type_name': 'Стиральные машины'},
 {'product_url': 'https://shop.by/stiralnye_mashiny/atlant_sma_60u1010/',
  'product_name': 'Стиральная машина ATLANT СМА 60У1010-00',
  'product_position': '4',
  

In [1]:
# to do - 1. list page - add breadcrumbs
# 0 - начальный перебор сделать
# to do - how to find out max page number in 

In [3]:
url = 'https://shop.by/stiralnye_mashiny/?page_id=1'
shop_url = 'https://shop.by/stiralnye_mashiny/'
from scrapy.crawler import CrawlerProcess
from loguru import logger
import extruct
import pprint

ua = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
import scrapy
from scrapy.spiders import CrawlSpider, Rule
from bs4 import BeautifulSoup
import requests
from typing import Dict, List

class JsonldExtractor:
    '''
    Must be Jsonld
    Returns Name of web page proudct
    '''
    @staticmethod
    def page_product(jdata: Dict) -> str:
        '''Run at step 1(a) - get category name'''
        for el in jdata.get('json-ld'):
            if el.get('@type') == 'Product':
                return el.get('name')

    @staticmethod
    def get_jdata(body):
        return extruct.extract(body, syntaxes=['json-ld'])
                
                
class MicrodataExtractor:
    '''extract microdayta'''

    @staticmethod
    def get_mdata(body):
        return extruct.extract(body, syntaxes=['microdata'])
    
    @staticmethod
    def breadcrumbs(mdata:Dict) -> List[Dict[str, str]]:
        '''
        Must be Microdata
        Run at step 1(b) - get website structure
        '''
        website_structure = []
        for el in mdata['microdata']:
            if el.get('type') == 'http://schema.org/BreadcrumbList':
                _properties = el.get('properties')
                _items_list = _properties.get('itemListElement') 
                for _item in _items_list:
                    item_properties = {}
                    _item_properties = _item.get('properties')
                    item_properties['full_url'] = _item_properties.get('item')
                    item_properties['name'] = _item_properties.get('name')
                    item_properties['position'] = _item_properties.get('position')
                    website_structure.append(item_properties)
        return website_structure
        
    
    @staticmethod
    def item_list(mdata:Dict) -> List[Dict[str, str]]:
        ''' 
        Must be Microdata
        Run as step 1(c) - get item list
        
        :param mdata = extuct.extract Dict
        :returns results List 
        '''
        results = []
        for el in mdata['microdata']:
            if el.get('type') == 'https://schema.org/ItemList':
                _items = el['properties'].get('itemListElement')
                for item in _items:
                    if item.get('type') == 'https://schema.org/ListItem':
                        item_features = {}  
                        item_properties = item.get('properties')
                        rel_item_url = item_properties.get('url')
                        item_features['full_url'] = base_url+rel_item_url.rstrip('#shop')
                        item_features['name'] = item_properties.get('name')
                        item_features['position'] = item_properties.get('position')
                        # item_description = item_properties.get('description')
                        results.append(item_features)
        return results
    
    @staticmethod
    def product(mdata:Dict) -> Dict[str, str]:
        ''' 
        Run as step 2 - get product details
    
        :param mdata = extuct.extract Dict
        :returns item_features Dict 
        '''
        
        for el in mdata['microdata']:
            if el.get('type') == 'https://schema.org/Product':
                item_features = {}  # to put in sql table
                _item_properties = el['properties']
                item_features['name'] = _item_properties.get('name')
                item_features['full_url'] = base_url + _item_properties.get('url')
                _offers_properties = _item_properties.get('offers').get('properties')
                item_features['offer_count'] = _offers_properties.get('offerCount')
                item_features['min_price'] = _offers_properties.get('lowPrice')
                _additional_properties = _item_properties.get('additionalProperty')
                for ap_dict in _additional_properties:
                    kv = ap_dict.get('properties')
                    key = kv.get('name')
                    val = kv.get('value')
                    item_features[key] = val
        return item_features

# to do

from scrapy.exceptions import CloseSpider

class ItemListSpider(CrawlSpider):
    name = "item_list"
    
    def __init__(self, urls_file, *a, **kw):
        super(ItemListSpider, self).__init__(*a, **kw)
        self.page_product = []
        self.breadcrumbs = []
        self.item_list = []
        
    def start_requests(self):
        urls = [
            f'https://shop.by/stiralnye_mashiny/?page_id={i}' for i in range (1,2)
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
            
    def parse(self, response):
        if response.status == 404: 
            raise CloseSpider('Recieve 404 response')

        jdata = JsonldExtractor.get_jdata(response.body)
        mdata = MicrodataExtractor.get_mdata(response.body)
        
        page_product = JsonldExtractor.page_product(jdata)
        breadcumbs = MicrodataExtractor.breadcrumbs(mdata)
        item_list = MicrodataExtractor.item_list(mdata)
        
        self.page_product.append(page_product)
        self.breadcumbs.append(breadcumbs)
        self.item_list.append(item_list)
        


class ProductSpider(CrawlSpider):
    name = 'product'

    def __init__(self, urls_file, *a, **kw):
        super(ProductSpider, self).__init__(*a, **kw)
        self.products = []
        
    def start_requests(self):
        urls = [
            'https://shop.by/stiralnye_mashiny/atlant_sma_60u1214_01'
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
            
    def parse(self, response):
        if response.status == 404: 
            raise CloseSpider('Recieve 404 response')

        mdata = MicrodataExtractor.get_mdata(response.body)
        product = MicrodataExtractor.product(mdata)
        self.products.append(product)



c = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36',
    'FEED_FORMAT': 'csv',
    'FEED_URI': 'output.csv',
    'DEPTH_LIMIT': 2,
    'CLOSESPIDER_PAGECOUNT': 3,
})
r1 = c.crawl(ProductSpider, urls_file='input.txt')
r2 = c.start()

class CustomCrawler:
    def __init__(self):
        self.output = None
        self.process = CrawlerProcess(get_project_settings(), )

    def yield_output(self, data):
        self.output = data

    def crawl(self, cls):
        self.process.crawl(cls, args={'callback': self.yield_output})
        self.process.start()


def crawl_static(cls):
    crawler = CustomCrawler()
    crawler.crawl(cls)
    return crawler.output



if __name__ == '__main__':

    
    output_filepath = os.path.join(ROOT_PROJECT, 'svetilnik/output')
    if not os.path.exists(output_filepath):
        os.mkdir(output_filepath)
    logging.getLogger(__name__).setLevel(logging.CRITICAL)
    process = CrawlerProcess(get_project_settings())

    out = crawl_static(DesignlampSpider)
    with open(os.path.join(output_filepath, 'designlamp.pkl'), 'wb') as f:
        pickle.dump(out, f)
    logger.info(len(out))

2024-04-07 12:31:57 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
2024-04-07 12:31:57 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0], pyOpenSSL 23.3.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.7, Platform Linux-6.5.0-26-generic-x86_64-with-glibc2.35
2024-04-07 12:31:57 [scrapy.addons] INFO: Enabled addons:
[]
2024-04-07 12:31:57 [py.warnings] WARNING: /home/amstel/miniconda3/envs/langchain/lib/python3.10/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the docum

NameError: name 'os' is not defined

In [37]:
base_url = 'https://shop.by'

In [4]:
with open("quotes-stiralnye_mashiny.html") as fp:
    soup = BeautifulSoup(fp, "html.parser")

In [51]:
with open("q-atlant_sma_60u1214_01.html") as fp:
    soup = BeautifulSoup(fp, "html.parser")

In [5]:
jdata = extruct.extract(soup.prettify(),  syntaxes=['json-ld'])
mdata = extruct.extract(soup.prettify(),  syntaxes=['microdata'])

In [12]:
extruct.extract(soup.prettify(),  ).keys()

{'microdata': [{'type': 'https://schema.org/WebPage',
   'value': 'Вся Беларусь Антополь Барановичи Барань Бегомль Белоозёрск Белыничи Берёза Березино Берёзовка Березовка Бобруйск Борисов Боровуха Браслав Брест Буда-Кошелёво Быхов Василевичи Верхнедвинск Ветка Вилейка Витебск Волковыск Воложин Высокое Ганцевичи Глубокое Глуск Гомель Горки Городея Городок Гродно Давид-Городок Дзержинск Дисна Добруш Докшицы Дрогичин Дубровно Дятлово Ельск Жабинка Житковичи Жлобин Жодино Заславль Зельва Иваново Ивацевичи Ивье Калинковичи Каменец Кировск Клецк Климовичи Кличев Кобрин Копыль Коссово Костюковичи Костюковка Коханово Красносельский Кричев Круглое Крупки Лельчицы Лепель Лида Логойск Логойск Лунин Лунинец Любань Ляховичи Малорита Марьина Горка Микашевичи Минск Миоры Мир Могилев Мозырь Молодечно Мосты Мстиславль Мядель Наровля Несвиж Новогрудок Новолукомль Новополоцк Орша Осиповичи Островец Ошмяны Паричи Петриков Пинск Плещеницы Полоцк Поставы Пружаны Радошковичи Раков Речица Рогачёв Россь Светло

In [79]:
page_product_name

'Стиральные машины'

In [75]:
mdata

{'json-ld': [{'@context': 'http://schema.org',
   '@type': 'Product',
   'name': 'Стиральные машины',
   'image': '/section/644.jpg',
   'description': 'Купить стиральную машину (стиралку) в каталоге 🎁Shop.by. ✔️Большой выбор, %скидки%. 💲Доступные цены. 🚚 Доставка по Минску и всей Беларуси.',
   'offers': {'@type': 'AggregateOffer',
    'highPrice': '18475.27',
    'lowPrice': '120',
    'offerCount': '3794',
    'priceCurrency': 'BYN'}}]}

In [61]:
type(soup.prettify())

str

In [97]:
website_structure = []

In [6]:
for _item in _items_list:
    _item_properties = _item.get('properties')
    item_properties['full_url'] = _item_properties.get('item')
    item_properties['name'] = _item_properties.get('name')
    item_properties['position'] = _item_properties.get('position')
    website_structure.append(item_properties)

NameError: name '_items_list' is not defined

In [100]:
website_structure

[{'full_url': '/', 'name': 'Главная', 'position': '0'},
 {'full_url': 'https://shop.by/bytovaya_tehnika/',
  'name': 'Бытовая техника',
  'position': '1'},
 {'full_url': 'https://shop.by/bytovaya_tehnika/tehnika_dlya_doma/',
  'name': 'Техника для дома',
  'position': '2'},
 {'full_url': '#', 'name': 'Стиральные машины', 'position': '3'}]

In [63]:
item_list = Extractor.item_list(mdata)

In [55]:
%%time
item_details = Extractor.product(mdata)

In [56]:
item_details

{'name': 'Стиральная машина ATLANT СМА 60У1214-01',
 'full_url': 'https://shop.by/stiralnye_mashiny/atlant_sma_60u1214_01/',
 'offer_count': '41',
 'min_price': '650.00',
 'Производитель': 'ATLANT',
 'Тип загрузки': 'Фронтальная',
 'Максимальная загрузка белья, кг': '6',
 'Тип машины': 'Барабанного типа',
 'Глубина, см': '40.6',
 'Ширина, см': '59.6',
 'Высота, см': '84.6',
 'Вес, кг': '62',
 'Степень автоматизации': 'Автомат',
 'Количество программ': '18',
 'Дисплей': 'Есть',
 'Установка': 'Отдельно стоящая',
 'Цвет': 'Белый',
 'Класс стирки': 'A',
 'Расход воды за стирку, л': '50',
 'Возможность дозагрузки белья': 'Есть',
 'Дополнительное полоскание': 'Есть',
 'Максимальное количество оборотов отжима, об/мин': '1200',
 'Класс отжима': 'B',
 'Выбор скорости отжима': 'Есть',
 'Отмена отжима': 'Есть',
 'Программа «легкая глажка»': 'Есть',
 'Сушка': 'Нет',
 'Класс энергопотребления': 'A+++',
 'Прямой привод (direct drive)': 'Нет',
 'Материал бака': 'Пластик',
 'Дополнительные функции': '

In [18]:
results

[{'name': 'Стиральная машина ATLANT СМА 60У1214-01',
  'full_url': 'https://shop.by/stiralnye_mashiny/atlant_sma_60u1214_01/',
  'offer_count': '41',
  'min_price': '650.00',
  'Производитель': 'ATLANT',
  'Тип загрузки': 'Фронтальная',
  'Максимальная загрузка белья, кг': '6',
  'Тип машины': 'Барабанного типа',
  'Глубина, см': '40.6',
  'Ширина, см': '59.6',
  'Высота, см': '84.6',
  'Вес, кг': '62',
  'Степень автоматизации': 'Автомат',
  'Количество программ': '18',
  'Дисплей': 'Есть',
  'Установка': 'Отдельно стоящая',
  'Цвет': 'Белый',
  'Класс стирки': 'A',
  'Расход воды за стирку, л': '50',
  'Возможность дозагрузки белья': 'Есть',
  'Дополнительное полоскание': 'Есть',
  'Максимальное количество оборотов отжима, об/мин': '1200',
  'Класс отжима': 'B',
  'Выбор скорости отжима': 'Есть',
  'Отмена отжима': 'Есть',
  'Программа «легкая глажка»': 'Есть',
  'Сушка': 'Нет',
  'Класс энергопотребления': 'A+++',
  'Прямой привод (direct drive)': 'Нет',
  'Материал бака': 'Пластик'

In [26]:
# itemlist

base_url='https://shop.by'
mdata = extruct.extract(soup.prettify(),  syntaxes=['microdata'])

                

In [45]:
import os

In [50]:
item_url

'shop.by/stiralnye_mashiny/atlant_sma_60u1214_01/#shop'

In [1]:
from bs4 import BeautifulSoup
from typing import Dict
from extruct.w3cmicrodata import MicrodataExtractor
from extruct.microformat import MicroformatExtractor
import pprint

def parse_product(el) -> Dict[str, str]:  
    '''return product details from microdata'''
    product_category = el.get('properties').get('category')
    product_brand = el.get('properties').get('brand')
    product_name = el.get('properties').get('name')
    product_url = el.get('properties').get('url')
    product_image_url = el.get('properties').get('image')
    _aggredagate_rating = el.get('properties').get('aggregateRating').get('properties')
    product_rating_value = _aggredagate_rating.get('ratingValue')
    product_best_rating = _aggredagate_rating.get('bestRating')
    product_worst_rating = _aggredagate_rating.get('worstRating')
    product_rating_count = _aggredagate_rating.get('ratingCount')
    product_review_count = _aggredagate_rating.get('reviewCount')
    return {
        'product_category':product_category,
        'product_brand':product_brand,
        'product_name':product_name,
        'product_url':product_url,
        'product_image_url':product_image_url,
        'product_rating_value':product_rating_value,
        'product_best_rating':product_best_rating,
        'product_worst_rating':product_worst_rating,
        'product_rating_count':product_rating_count,
        'product_review_count':product_review_count,
    }

def parse_reviews(el) -> str:
    '''returns reviews delimited by /n from microdata'''
    total_reviews = []
    _reviews = el.get('properties').get('review')
    for review in _reviews:
        review_date_published = review.get('properties').get('datePublished')
        review_description = review.get('properties').get('description')
        _review_rating_properties = review.get('properties').get('reviewRating').get('properties')
        review_ratng_value = _review_rating_properties.get('ratingValue')
        review_best_rating = _review_rating_properties.get('bestRating')
        total_reviews.append(review_description)
    return '\n\n'.join(total_reviews)

In [14]:
!ls

duck_google_yandex.ipynb  extract_item_names.py  reviews_res.html
duck_res.html		  langchain_duck.py	 scrapy_onliner
example2.png		  output.csv		 search_duck.py
example3.png		  parse_reviews.py	 summarize_reviews_step1.py
example.png		  product_res.html	 wah_spider.html


In [19]:
import extruct


In [20]:

pp = pprint.PrettyPrinter(indent=2)
# mde = MicrodataExtractor()
data = extruct.extract(soup.prettify(), syntaxes=['microdata'])
pp.pprint(data)

{ 'microdata': [ { 'type': 'https://schema.org/WebPage',
                   'value': 'Вся Беларусь Антополь Барановичи Барань Бегомль '
                            'Белоозёрск Белыничи Берёза Березино Берёзовка '
                            'Березовка Бобруйск Борисов Боровуха Браслав Брест '
                            'Буда-Кошелёво Быхов Василевичи Верхнедвинск Ветка '
                            'Вилейка Витебск Волковыск Воложин Высокое '
                            'Ганцевичи Глубокое Глуск Гомель Горки Городея '
                            'Городок Гродно Давид-Городок Дзержинск Дисна '
                            'Добруш Докшицы Дрогичин Дубровно Дятлово Ельск '
                            'Жабинка Житковичи Жлобин Жодино Заславль Зельва '
                            'Иваново Ивацевичи Ивье Калинковичи Каменец '
                            'Кировск Клецк Климовичи Кличев Кобрин Копыль '
                            'Коссово Костюковичи Костюковка Коханово '
                     

In [20]:
data.keys()

dict_keys(['microdata'])

In [27]:
pprint.pprint(item)

{'properties': {'description': 'Отдельно стоящая, барабанного типа, глубина '
                               '41.5 см, загрузка фронтальная, 6 кг, '
                               'количество программ 15, класс '
                               'энергопотребления А++, материал бака пластик, '
                               'отложенный старт, обработка паром, индикация '
                               'ошибок, звуковой сигнал, защита от детей, '
                               'контроль дисбаланса, контроль пенообразования, '
                               'ширина 60 см.',
                'image': {'properties': {'image': '/images/a_beko_wsre6512zaa_icon.webp'},
                          'type': 'http://schema.org/ImageObject'},
                'name': 'Стиральная машина BEKO WSRE6512ZAA',
                'position': '1',
                'url': '/stiralnye_mashiny/beko_wsre6512zaa/#shop'},
 'type': 'https://schema.org/ListItem'}


In [25]:
fields = {
  "standalone": "standalone",
  "drum_type": "drum_type",
  "depth": "depth",
  "front_loading": "front_loading",
  "weight": "weight",
  "number_of_programs": "number_of_programs",
  "energy_class": "energy_class",
  "material_of_tub": "material_of_tub",
  "delayed_start": "delayed_start",
  "steam_generation": "steam_generation",
  "error_indication": "error_indication",
  "sound_signal": "sound_signal",
  "child_protection": "child_protection",
  "balance_control": "balance_control",
  "foam_control": "foam_control",
  "width": "width"
}


In [ ]:
get

In [26]:
{k:v.strip(' ') for k,v in zip(fields.keys(), item_description.split(','))}

{'standalone': 'Отдельно стоящая',
 'drum_type': 'барабанного типа',
 'depth': 'глубина 41.5 см',
 'front_loading': 'загрузка фронтальная',
 'weight': '6 кг',
 'number_of_programs': 'количество программ 15',
 'energy_class': 'класс энергопотребления А++',
 'material_of_tub': 'материал бака пластик',
 'delayed_start': 'отложенный старт',
 'steam_generation': 'обработка паром',
 'error_indication': 'индикация ошибок',
 'sound_signal': 'звуковой сигнал',
 'child_protection': 'защита от детей',
 'balance_control': 'контроль дисбаланса',
 'foam_control': 'контроль пенообразования',
 'width': 'ширина 60 см.'}

In [24]:
item_description

'Отдельно стоящая, барабанного типа, глубина 41.5 см, загрузка фронтальная, 6 кг, количество программ 15, класс энергопотребления А++, материал бака пластик, отложенный старт, обработка паром, индикация ошибок, звуковой сигнал, защита от детей, контроль дисбаланса, контроль пенообразования, ширина 60 см.'

In [28]:
i = 0
for el in data:
    if el.get('type') == 'https://schema.org/Product':
        if 'properties' in el:
            product_details = parse_product(el)
            product_reviews = parse_reviews(el)
            i+=1

In [40]:
product_details

{'product_category': 'Мобильные телефоны',
 'product_brand': 'Xiaomi',
 'product_name': 'Смартфон Xiaomi 14',
 'product_url': 'https://market.yandex.by/product--smartfon-xiaomi-14/1943226316/reviews',
 'product_image_url': 'https://avatars.mds.yandex.net/get-mpic/4818396/img_id4780156176612636343.jpeg/x332_trim',
 'product_rating_value': '5',
 'product_best_rating': '5',
 'product_worst_rating': '1',
 'product_rating_count': '45',
 'product_review_count': '20'}

In [42]:
from extruct.jsonld import JsonLdExtractor
from extruct.rdfa import RDFaExtractor
from extruct.dublincore import DublinCoreExtractor
import extruct
jslde = JsonLdExtractor()
rdfa = RDFaExtractor()
ex = DublinCoreExtractor()



data = extruct.extract(soup.prettify())
pp.pprint(data)


{ 'dublincore': [ { 'elements': [ { 'URI': 'http://purl.org/dc/elements/1.1/description',
                                    'content': 'Смартфон Xiaomi 14: отзывы '
                                               'покупателей на Яндекс Маркете. '
                                               'Достоинства и недостатки '
                                               'товара. Важная информация о '
                                               'товаре Смартфон Xiaomi 14: '
                                               'описание, фотографии, цены, '
                                               'варианты доставки, магазины на '
                                               'карте.',
                                    'name': 'description'}],
                    'namespaces': {},
                    'terms': []}],
  'json-ld': [],
  'microdata': [ { 'properties': {'url': 'http://market.yandex.by'},
                   'type': 'https://schema.org/WebSite'},
                 { 'properti

In [27]:
data.keys()

dict_keys(['microdata'])

In [25]:
reviews_url = 'https://market.yandex.ru/product--xiaomi-14/1943226316/reviews'

In [27]:
import re
from bs4 import BeautifulSoup

def search_term_share(
    search_term: str,
    result_text: str
):
    len_search = len(search_term)
    len_result = len(result_text)
    start_pos = result_text.find(search_term)
    if start_pos != -1:
        return len_search / len_result
    else:
        return 0.0
def clear_str(s):
    return s.replace("Стоит ли покупать", "").replace("? Отзывы на Яндекс Маркете", "").strip().lower()
    
with open("duck_res.html") as fp:
    soup = BeautifulSoup(fp, "html.parser")

first_result = soup.find("article", id='r1-0')
second_result = soup.find("article", id='r1-1')

first_result_text = first_result.find("h2").find("span").text
second_result_text = second_result.find("h2").find("span").text

search_term = 'Xiaomi 14'.lower()



first_result_text = clear_str(first_result_text).lower()
second_result_text = clear_str(second_result_text).lower()

first_result_text.find(search_term)



first_share = search_term_share(search_term, first_result_text)
second_share = search_term_share(search_term, second_result_text)
if first_share >= second_share:
    chosen_result = first_result
else: 
    chosen_result = second_result

In [22]:
second_result_text

'смартфон xiaomi 14'

In [85]:
from fastbm25 import fastbm25

corpus = [
   first_result_text,
   second_result_text
]
tokenized_corpus = [doc.lower().split(" ") for doc in corpus]
model = fastbm25(tokenized_corpus)
query = search_term.lower().split()
result = model.top_k_sentence(query,k=1)
print(result)

[(['смартфон', 'xiaomi', '14', 'pro'], 0, -0.56)]


In [86]:
first_result_text

'Смартфон Xiaomi 14 Pro'

In [88]:
model.similarity_bm25(first_result_text, second_result_text)

0.0

In [89]:
query

['xiaomi', '14']

In [ ]:
link = top_result.find("a", {'href': re.compile(r'reviews')}).get('href')
print(link)

https://market.yandex.ru/product--smartfon-xiaomi-14-pro/1943141568/reviews


In [58]:
link.attrs

AttributeError: 'str' object has no attribute 'attrs'

In [53]:
link

'https://market.yandex.ru/product--xiaomi-14/1943226316/reviews'

In [ ]:
{'href': re.compile(r'crummy\.com/')}

In [30]:
len(top_result)

3

In [24]:
top_result.find("a",)

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [22]:
type(top_result)

bs4.element.ResultSet

In [26]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

import requests 
from bs4 import BeautifulSoup

url = 'https://yandex.by/search/'
params={
        'text':'Xiaomi 14',
        'lr':157,
        'search_source':'yaby_desktop_common',
        'src':'suggest_B',
    }
for i, (k,v ) in enumerate(params.items()):
    if i == 0:
        prefix = '?'
    else:
        prefix = '&'
        url += prefix + str(k) + '=' + str(v).replace(' ', '+')

from playwright.sync_api import sync_playwright
playwright = sync_playwright().start()
browser = playwright.firefox.launch(headless=True)
page = browser.new_page()
page.goto(reviews_url)
content = page.content()

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [ ]:
with open('google_res.html', 'w') as f:
    f.write(response.text)

In [ ]:
page.screenshot(path="example2.png")
browser.close()
playwright.stop()

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [34]:
url += '&' + str(k).replace(' ', '+') + '=' + str(v) 

In [39]:
url


'https://yandex.by/search/&text=Xiaomi+14&lr=157&search_source=yaby_desktop_common&src=suggest_B'

NameError: name 'page' is not defined

In [17]:
from selenium import webdriver

driver = webdriver.Chrome()  # can be webdriver.Chrome()
driver.get("about:blank")

data = '<h1>test</h1>'  # supposed to come from BeautifulSoup
driver.execute_script('document.body.innerHTML = "{html}";'.format(html=soup))

JavascriptException: Message: javascript error: Invalid or unexpected token
  (Session info: chrome=120.0.6099.71)
Stacktrace:
#0 0x564f9f274f83 <unknown>
#1 0x564f9ef2dcf7 <unknown>
#2 0x564f9ef342a3 <unknown>
#3 0x564f9ef36bb4 <unknown>
#4 0x564f9efc6ba3 <unknown>
#5 0x564f9efa70b2 <unknown>
#6 0x564f9efc6006 <unknown>
#7 0x564f9efa6e53 <unknown>
#8 0x564f9ef6edd4 <unknown>
#9 0x564f9ef701de <unknown>
#10 0x564f9f239531 <unknown>
#11 0x564f9f23d455 <unknown>
#12 0x564f9f225f55 <unknown>
#13 0x564f9f23e0ef <unknown>
#14 0x564f9f20999f <unknown>
#15 0x564f9f262008 <unknown>
#16 0x564f9f2621d7 <unknown>
#17 0x564f9f274124 <unknown>
#18 0x787605494ac3 <unknown>


In [22]:
!pip install selenium

  Using cached typing_extensions-4.10.0-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 10.8 MB/s eta 0:00:00m eta 0:00:010:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 9.4 MB/s eta 0:00:006 MB/s eta 0:00:01
Using cached typing_extensions-4.10.0-py3-none-any.whl (33 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.0
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0


In [20]:
type(soup)

bs4.BeautifulSoup

In [ ]:
start=0
  &num=10
  &q=red+sox
  &cr=countryCA
  &lr=lang_fr
  &client=google-csbe
  &output=xml_no_dtd
  &cx=00255077836266642015:u-scht7a-8i

In [10]:

# response.text

In [13]:


headers_Get = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }


def google(q):
    s = requests.Session()
    q = '+'.join(q.split())
    url = 'https://www.google.com/search?q=' + q + '&ie=utf-8&oe=utf-8'
    r = s.get(url, headers=headers_Get)

    soup = BeautifulSoup(r.text, "html.parser")
    output = []
    for searchWrapper in soup.find_all('h3', {'class':'r'}): #this line may change in future based on google's web page structure
        url = searchWrapper.find('a')["href"] 
        text = searchWrapper.find('a').text.strip()
        result = {'text': text, 'url': url}
        output.append(result)

    return output

In [14]:
output = google('Xiaomi 14')

In [15]:
output

[]

In [35]:
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool

search = GoogleSearchAPIWrapper()

tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)

ValidationError: 1 validation error for GoogleSearchAPIWrapper
__root__
  Did not find google_api_key, please add an environment variable `GOOGLE_API_KEY` which contains it, or pass `google_api_key` as a named parameter. (type=value_error)

In [25]:
import requests

In [26]:
user_query = 'яндекс маркет отзывы Xiaomi 14'
query = f'https://www.google.com/search?q={user_query}'

In [27]:
results = requests.get(query)

In [20]:
search = DuckDuckGoSearchAPIWrapper(region="ru-ru", time="d", max_results=20, backend='html')
# searching_tool = DuckDuckGoSearchResults(api_wrapper=search,)

In [21]:
result = searching_tool.invoke('яндекс маркет отзывы')

In [22]:
search_run = DuckDuckGoSearchRun()

In [11]:
result = search_run.invoke('яндекс маркет отзывы')

In [14]:
search.invoke('яндекс маркет отзывы')

AttributeError: 'DuckDuckGoSearchAPIWrapper' object has no attribute 'invoke'

In [23]:
search.run(query='яндекс маркет отзывы Xiaomi 14')

'В DNS — 65 999 ₽ на «Яндекс Маркете» — от 50 959 ₽ ... тесты коллег-журналистов и отзывы пользователей, мы отобрали лучшие ноутбуки в бюджете до 70 000 ₽. ... Xiaomi Redmi Book Pro 14. Asus VivoBook 17 X1704ZA-AU121W. Изучаете отзывы про ТОП—7. Лучшие ноутбуки 13-14 дюймов. Рейтинг 2023 года!? Почитайте свежие реальные отзывы от имени бывших сотрудников на сайте shitcompany.org Стоит ли покупать Xiaomi Poco F5 Pro зимой 2023-2024 года? / Арстайл / - тема важная и интересная, поэтому наша редакция подготовила подробный разбор этой темы на сайте mnogorabotnikov.ru Лучшие смартфоны без ШИМ: Топ-5 смартфонов с IPS-экраном 📱 Рейтинг 2024 года - тема важная и интересная, поэтому наша редакция подготовила подробный разбор этой темы на сайте mnogorabotnikov.ru Изучаете отзывы про ТОП-6. Лучшие ноутбуки для работы и учебы ⚡ Рейтинг 2024 года по цене-качеству? Почитайте свежие реальные отзывы от имени бывших сотрудников на сайте shitcompany.org От 5 до 10 тысяч рублей выручают 16% респондентов,